In [1]:
import itertools
import re

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# customisations
pd.set_option("max_columns", 30)
plt.style.use('default')
plt.rcdefaults()
# sns.set() # default seaborn style

*Needed some data optimization*
- Splitting up the genres/categories
- keeping the lower end of the owners range
- calculate the rating based on the steam method 
- creating a column for paid/free game
- creating a release year column



In [6]:
def calc_rating(row):
    import math
    pos = row['positive_ratings']
    neg = row['negative_ratings']

    total_rev = pos + neg
    average = pos / total_rev
    
    score = average - (average*0.5) * 2**(-math.log10(total_reviews + 1)) #Using the Steam Rating Formula

    return score * 100


def get_unique(series):
    #splitting
    return set(list(itertools.chain(*series.apply(lambda x: [c for c in x.split(';')]))))


def process_cat_gen_tag(df):
    
    cat_cols = get_unique(df['categories']) # get all unique category names
    
    cat_cols = [
        'Multi-player',
        'Single-player']
    
    # splitting categories
    for col in sorted(cat_cols):
        col_name = re.sub(r'[\s\-\/]', '_', col.lower())
        col_name = re.sub(r'[()]', '', col_name)
        
        df[col_name] = df['categories'].apply(lambda x: 1 if col in x.split(';') else 0)
        
    
    gen_cols = get_unique(df['genres'])
    
    
    gen_cols = [
        'Action',
        'Adventure',
        'Casual',
        'Indie',
        'Massively Multiplayer',
        'RPG',
        'Racing',
        'Simulation',
        'Sports',
        'Strategy']
    
    gen_col_names = []
    
    
    for col in sorted(gen_cols):
        col_name = col.lower().replace('&', 'and').replace(' ', '_')
        gen_col_names.append(col_name)
        
        df[col_name] = df['genres'].apply(lambda x: 1 if col in x.split(';') else 0)
       
    # this removes software (zeros in the newly created genre columns)        
    gen_sums = df[gen_col_names].sum(axis=1)
    df = df[gen_sums > 0].copy()
        
        
    # remove redundant columns
    df = df.drop(['categories', 'steamspy_tags'], axis=1)
    
    return df


def pre_process():
    
    df = pd.read_csv('steam.csv')
        
    # keep windows only (small amount of other platform games in file)
    df = df[df['platforms'].str.contains('windows')].drop('platforms', axis=1).copy()
    
    df['owners'] = df['owners'].str.split('-').apply(lambda x: x[0]).astype(int)
    
    # Rating
    df['total_ratings'] = df['positive_ratings'] + df['negative_ratings']
    df['rating_ratio'] = df['positive_ratings'] / df['total_ratings']
    df['rating'] = df.apply(calc_rating, axis=1)
    
    # Datetime operation
    df['release_date'] = df['release_date'].astype('datetime64[ns]')
    df['release_year'] = df['release_date'].apply(lambda x: x.year)
    
    # splitting genres and categories
    df = process_cat_gen_tag(df)
    
    return df



In [7]:
df1 = pre_process()

df1.head(20)

Verify no missing values:
0    30
dtype: int64


,appid,name,release_date,developer,publisher,required_age,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year,multi_player,single_player,action,adventure,casual,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,Action,0,124534,3339,17612,317,10000000,7.19,127873,0.973888,95.975678,2000,1,0,1,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,Action,0,3318,633,277,62,5000000,3.99,3951,0.839787,80.508259,1999,1,0,1,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,Action,0,3416,398,187,34,5000000,3.99,3814,0.895648,85.823915,2003,1,0,1,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,Action,0,1273,267,258,184,5000000,3.99,1540,0.826623,78.126542,2001,1,0,1,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,Action,0,5250,288,624,415,5000000,3.99,5538,0.947996,91.260480,1999,1,1,1,0,0,0,0,0,0,0,0,0
5,60,Ricochet,2000-11-01,Valve,Valve,0,Action,0,2758,684,175,10,5000000,3.99,3442,0.801278,76.676163,2000,1,0,1,0,0,0,0,0,0,0,0,0
6,70,Half-Life,1998-11-08,Valve,Valve,0,Action,0,27755,1100,1300,83,5000000,7.19,28855,0.961878,94.002967,1998,1,1,1,0,0,0,0,0,0,0,0,0
7,80,Counter-Strike: Condition Zero,2004-03-01,Valve,Valve,0,Action,0,12120,1439,427,43,10000000,7.19,13559,0.893871,86.838469,2004,1,1,1,0,0,0,0,0,0,0,0,0
8,130,Half-Life: Blue Shift,2001-06-01,Gearbox Software,Valve,0,Action,0,3822,420,361,205,5000000,3.99,4242,0.900990,86.454395,2001,0,1,1,0,0,0,0,0,0,0,0,0
9,220,Half-Life 2,2004-11-16,Valve,Valve,0,Action,33,67902,2419,691,402,10000000,7.19,70321,0.965601,94.882618,2004,0,1,1,0,0,0,0,0,0,0,0,0


free/paid column add

In [9]:
df1['type'] ='Free'
df1.loc[df1['price'] > 0, 'type'] = 'Paid'

In [12]:
df1 = df1.assign(year=df1['release_date'].dt.year)
df1

,appid,name,release_date,developer,publisher,required_age,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year,multi_player,single_player,action,adventure,casual,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy,type,year
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,Action,0,124534,3339,17612,317,10000000,7.19,127873,0.973888,95.975678,2000,1,0,1,0,0,0,0,0,0,0,0,0,Paid,2000
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,Action,0,3318,633,277,62,5000000,3.99,3951,0.839787,80.508259,1999,1,0,1,0,0,0,0,0,0,0,0,0,Paid,1999
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,Action,0,3416,398,187,34,5000000,3.99,3814,0.895648,85.823915,2003,1,0,1,0,0,0,0,0,0,0,0,0,Paid,2003
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,Action,0,1273,267,258,184,5000000,3.99,1540,0.826623,78.126542,2001,1,0,1,0,0,0,0,0,0,0,0,0,Paid,2001
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,Action,0,5250,288,624,415,5000000,3.99,5538,0.947996,91.260480,1999,1,1,1,0,0,0,0,0,0,0,0,0,Paid,1999
5,60,Ricochet,2000-11-01,Valve,Valve,0,Action,0,2758,684,175,10,5000000,3.99,3442,0.801278,76.676163,2000,1,0,1,0,0,0,0,0,0,0,0,0,Paid,2000
6,70,Half-Life,1998-11-08,Valve,Valve,0,Action,0,27755,1100,1300,83,5000000,7.19,28855,0.961878,94.002967,1998,1,1,1,0,0,0,0,0,0,0,0,0,Paid,1998
7,80,Counter-Strike: Condition Zero,2004-03-01,Valve,Valve,0,Action,0,12120,1439,427,43,10000000,7.19,13559,0.893871,86.838469,2004,1,1,1,0,0,0,0,0,0,0,0,0,Paid,2004
8,130,Half-Life: Blue Shift,2001-06-01,Gearbox Software,Valve,0,Action,0,3822,420,361,205,5000000,3.99,4242,0.900990,86.454395,2001,0,1,1,0,0,0,0,0,0,0,0,0,Paid,2001
9,220,Half-Life 2,2004-11-16,Valve,Valve,0,Action,33,67902,2419,691,402,10000000,7.19,70321,0.965601,94.882618,2004,0,1,1,0,0,0,0,0,0,0,0,0,Paid,2004


In [13]:
len(df1)

25951

In [14]:
df1['developer'].value_counts()

Choice of Games                          94
Ripknot Systems                          62
Laush Dmitriy Sergeevich                 51
Nikita "Ghost_RUS"                       49
Dexion Games                             45
RewindApp                                43
Hosted Games                             42
Blender Games                            40
Humongous Entertainment                  36
For Kids                                 36
EnsenaSoft                               35
Eipix Entertainment                      32
Warfare Studios                          31
HeR Interactive                          30
Nekyau Games                             30
MumboJumbo                               29
ERS G Studios                            28
Creobit                                  28
HexWar Games                             27
Sly                                      26
Milestone S.r.l.                         25
DRUNKEN APES                             25
Valve                           

In [15]:
df1['publisher'].value_counts()

Big Fish Games                           212
Strategy First                           135
Ubisoft                                  111
THQ Nordic                                97
Square Enix                               96
Sekai Project                             95
Choice of Games                           94
1C Entertainment                          88
Dagestan Technology                       87
SEGA                                      78
Slitherine Ltd.                           71
Plug In Digital                           71
KISS ltd                                  69
Artifex Mundi                             67
Buka Entertainment                        66
Devolver Digital                          65
AGM PLAYISM                               65
Atriagames                                63
Ripknot Systems                           62
Degica                                    61
BANDAI NAMCO Entertainment                56
Focus Home Interactive                    55
Paradox In

In [17]:
df1.to_pickle('steamop.pkl')